In [10]:
from binance import Client
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import schedule
import math
import scipy.stats as st
import ta
import statistics
import smtplib
import os
import time
from time import sleep
import re
import pandas as pd
import matplotlib.pyplot as plt
import pandas_ta as ta
import numpy as np
import datetime

api_key="pTZlrkaCbTEoa17nor24Tw53OWYFlMBQIOmaftSLcjg5lASfhZ0048zcxOZLrBgN"
api_secret="PxcyhPku5uQon1krBa3glze2sF7oZQ3gqb9GRVkh25OFvCtyAkML139l4z9d9BwG"
client=Client(api_key,api_secret)
client.get_account()

{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'brokered': False,
 'updateTime': 1664245146355,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00939456', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.00515877', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.05242887', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.05663872', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '4

In [2]:
! dir

 El volumen de la unidad C es Windows-SSD
 El n£mero de serie del volumen es: 60A9-EF9D

 Directorio de C:\Users\57314\Binance Automation

14/10/2022  03:34 p.ÿm.    <DIR>          .
14/10/2022  10:13 p.ÿm.    <DIR>          ..
15/09/2022  06:22 p.ÿm.    <DIR>          .ipynb_checkpoints
03/04/2022  11:31 p.ÿm.            55.600 Automated trading.ipynb
27/09/2022  08:59 p.ÿm.            95.204 BTC3min.ipynb
14/10/2022  03:33 p.ÿm.            99.929 btc5min.ipynb
12/06/2022  07:46 p.ÿm.         1.491.751 D1_Volume_Strategy.ipynb
14/10/2022  03:34 p.ÿm.           112.067 ETH3MIN.ipynb
20/07/2022  12:49 p.ÿm.            90.549 Getting Info.ipynb
24/06/2022  07:55 a.ÿm.            49.719 Keeping eye-Master-Copy2.ipynb
21/06/2022  10:53 p.ÿm.            54.984 Scalping (BNB) Master-Copy3.ipynb
18/08/2022  12:28 p.ÿm.            60.840 SOL DailyTrading .ipynb
27/06/2022  12:23 a.ÿm.            98.545 trading v1.ipynb
17/07/2022  12:16 a.ÿm.           104.121 Trading V2.ipynb
04/08/2022  08:3

In [11]:
def getdata(symbol, interval, lookback):
    frame=pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame=frame.iloc[:,:6]
    frame.columns=['Time','Open','High','Low', 'Close','Volume']
    frame=frame.set_index('Time')
    frame.index=pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    frame['rsi']=ta.momentum.rsi(frame.Close, window=14)
    frame["EMA_20"] = ta.ema(frame.Close, length=20)
    frame["Close Lag 5"]=frame['Close'].shift(5, axis=0)
    frame["Return 5m"]=(frame['Close']-frame['Close Lag 5'])/frame['Close Lag 5']
    frame['CRYPTO']=symbol
    frame['Date_time'] = frame.index
    frame['Height']=abs(frame['High']-frame['Low'])
    frame['Vol std'] = frame['Volume'].rolling(50).std()
    return frame

def check_decimals(symbol):
    info = client.get_symbol_info(symbol)
    val = info['filters'][2]['stepSize']
    decimal = 0
    is_dec = False
    for c in val:
        if is_dec is True:
            decimal += 1
        if c == '1':
            break
        if c == '.':
            is_dec = True
    return decimal

In [12]:
path='C:\\Users\\57314\\OneDrive - Universidad de los Andes\\Binance Automation\\DailyTrading.xlsx'
trading_df=pd.read_excel(path)

In [13]:
cry='BTCUSDT'
qtyUSD=19
buyqty=19
qtybtc=0.00100000
initial=datetime.datetime.now()
timeRun=0
buyprice=0
end=False
sell=False
while timeRun<40000 and end==False:
    time.sleep(20)
    final=datetime.datetime.now()
    timeRun=(final-initial).seconds
    df_1=getdata(cry,'3m','2 h ago UTC')
    macd_df=df_1.ta.macd(close='close', fast=3, slow=12, signal=15, append=True)
    price=df_1.Close.iloc[-1]
    df_1['MACD']=macd_df[macd_df.columns[0]]
    df_1['MACD_S']=macd_df[macd_df.columns[2]]
    df_1['Price_std'] = df_1['Close'].rolling(20).std()
    std=df_1.Price_std.iloc[-1]
    dec=check_decimals(cry)
    qty = round(qtyUSD / price, dec)
    qtybuy = round(buyqty / price, dec)
    gain=0
    maxGain=0
    rangeTime= (final-trading_df.iloc[-1]['date Transaction']).total_seconds() / 60.0
    if trading_df.iloc[-1]['Move']=='BUY':
        sell=False
        if pd.isna(trading_df.iloc[-1]['Max value']):
            if price > 1.002* trading_df.iloc[-1]['Crypto Value']:
                trading_df.at[len(trading_df)-1,'Max value']=price
                print('Max Value set '+ str(price))
        else:
            if price > trading_df.iloc[-1]['Max value']:
                trading_df.at[len(trading_df)-1,'Max value'] = price
                print(str(price))
            elif price > 1.001* trading_df.iloc[-1]['Crypto Value']:
                gain= price - trading_df.iloc[-1]['Crypto Value']
                maxGain=trading_df.iloc[-1]['Max value'] -trading_df.iloc[-1]['Crypto Value']
                if rangeTime<10:
                    print('Enter 1')
                    if gain< maxGain *0.65:
                        order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qtybtc)
                        qty=float(order['fills'][0]['qty'])
                        buyprice=float(order['fills'][0]['price'])
                        amountUSD=round(qty*buyprice, 3)
                        trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                        print('Enter Sell 1: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))
                elif rangeTime<60:
                    print('enter 2')
                    if gain< maxGain *0.8:
                            if df_1.iloc[-1]['MACD']<0 and df_1.iloc[-2]['MACD']<df_1.iloc[-3]['MACD']:
                                order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qtybtc)
                                qty=float(order['fills'][0]['qty'])
                                buyprice=float(order['fills'][0]['price'])
                                amountUSD=round(qty*buyprice, 3)
                                trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                                print('Enter Sell 2: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))
                            elif gain< maxGain *0.7 and maxGain>std:
                                order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qtybtc)
                                qty=float(order['fills'][0]['qty'])
                                buyprice=float(order['fills'][0]['price'])
                                amountUSD=round(qty*buyprice, 3)
                                trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                                print('Enter Sell 4: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))
                            elif gain< maxGain *0.5:
                                order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qtybtc)
                                qty=float(order['fills'][0]['qty'])
                                buyprice=float(order['fills'][0]['price'])
                                amountUSD=round(qty*buyprice, 3)
                                trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                                print('Enter Sell 5: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))
                else:
                    if gain< maxGain *0.85:
                        print('enter 3')
                        df_5=getdata(cry,'5m','3 h ago UTC')
                        macd_df5=df_5.ta.macd(close='close', fast=3, slow=15, signal=20, append=True)
                        df_5['MACD']=macd_df5[macd_df5.columns[0]]
                        print(df_5.iloc[-1]['MACD'])
                        if df_5.iloc[-1]['MACD']<0 and df_1.iloc[-1]['MACD']<0:
                            order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                            qty=float(order['fills'][0]['qty'])
                            buyprice=float(order['fills'][0]['price'])
                            amountUSD=round(qty*buyprice, 3)
                            trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                            print('Enter Sell 6: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))
                        elif gain< maxGain *0.85 and maxGain>2*std :
                            order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                            qty=float(order['fills'][0]['qty'])
                            buyprice=float(order['fills'][0]['price'])
                            amountUSD=round(qty*buyprice, 3)
                            trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                            print('Enter Sell 7: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))
                        elif gain< maxGain *0.7 :
                            order=client.create_order(symbol=cry,side='SELL', type= 'MARKET', quantity=qty)
                            qty=float(order['fills'][0]['qty'])
                            buyprice=float(order['fills'][0]['price'])
                            amountUSD=round(qty*buyprice, 3)
                            trading_df.loc[len(trading_df.index)] = ['SELL',cry, qty, amountUSD, buyprice,final,None]
                            print('Enter Sell 8: ' + str(gain) + "and Price: "+ str(buyprice) + str(final))

    elif trading_df.iloc[-1]['Move']=='SELL':
        buyprice=trading_df.iloc[-1]['Crypto Value']
        if price< (buyprice - std) and sell == False and df_1.iloc[-1]['MACD']< df_1.iloc[-1]['MACD_S']:
            sell=True
            print(sell)
        if price<buyprice:
            if (df_1.iloc[-2]['MACD']<0 and df_1.iloc[-1]['MACD']>0) and sell == True :
                order=client.create_order(symbol=cry,side='BUY', type= 'MARKET', quantity=qtybtc)
                qty=float(order['fills'][0]['qty'])
                buyprice=float(order['fills'][0]['price'])
                amountUSD=round(qty*buyprice, 3)
                trading_df.loc[len(trading_df.index)] = ['BUY',cry, qty, amountUSD, buyprice,final,None]
                print('Enter Buy 1: '  + "Price: "+ str(buyprice) + str(final))
            elif sell==True:
                print('Enter 2 Sell')
                if df_1.iloc[-1]['MACD']>df_1.iloc[-2]['MACD'] or df_1.iloc[-1]['MACD']> df_1.iloc[-1]['MACD_S']:
                    order=client.create_order(symbol=cry,side='BUY', type= 'MARKET', quantity=qtybtc)
                    qty=float(order['fills'][0]['qty'])
                    buyprice=float(order['fills'][0]['price'])
                    amountUSD=round(qty*buyprice, 3)
                    trading_df.loc[len(trading_df.index)] = ['BUY',cry, qty, amountUSD, buyprice,final,None]
                    print('Enter Buy 2 '  + "Price: "+ str(buyprice) + str(final))
            
                    
            

True
Enter 2 Sell
Enter 2 Sell
Enter 2 Sell
Enter 2 Sell
Enter 2 Sell
Enter Buy 2 Price: 19764.652022-09-26 21:40:36.076502
Max Value set 19831.01
enter 2
enter 2
19835.96
enter 2
enter 2
enter 2
19839.99
enter 2
19850.15
19861.31
enter 2
19864.1
19887.86
19932.94
19949.02
enter 2
19958.71
enter 2
enter 2
enter 2
19970.62
20052.81
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
enter 2
Enter Sell 4: 197.37999999999738and Price: 19964.572022-09-26 22:04:57.481363


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [ ]:
path='C:\\Users\\57314\\OneDrive - Universidad de los Andes\\Binance Automation\\DailyTrading.xlsx'
trading_df.to_excel(path, index=False)

In [ ]:
trading_df